# CS2 Demo Parsing — Round Processing
This notebook parses CS2 demos, computes round outcomes, derives time features, and filters out pre-freeze prologue ticks while keeping buy (freeze) and live phases. The flow:
- Imports and environment setup
- Paths to demos and assets
- Parse header and ticks
- Aggregate round winners/reasons
- Compute `seconds_elapsed_in_round`
- Drop ticks before freeze phase begins
- Map human-readable labels
- Preview results (optional training at the end)

# Imports and Constants

In [113]:
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import sys
sys.path.append("..")
from backend.func import DemoProcessing
import backend.constants as constants
from demoparser2 import DemoParser

import os

from tqdm import tqdm
from glob import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Unzip all possible files in demos folder

In [114]:
from glob import glob

# Try to extract RAR files if extraction tool is available
rar_files = glob(os.path.join(constants.DEMOS_DIR, "*.rar"))

if rar_files:
    print(f"Found {len(rar_files)} .rar files. Attempting extraction...")
    extracted = 0
    failed = 0
    for file in rar_files:
        try:
            import patoolib
            print(f"  Extracting: {os.path.basename(file)}")
            patoolib.extract_archive(file, outdir=constants.DEMOS_DIR, verbosity=-1)
            # Remove the .rar file after successful extraction
            os.remove(file)
            extracted += 1
            print(f"    ✓ Success")
        except Exception as e:
            failed += 1
            print(f"    ⚠ Failed (corrupted or unsupported): {str(e)[:50]}")
            # Try to remove the corrupted RAR file
            try:
                os.remove(file)
                print(f"    🗑️ Removed corrupted file")
            except:
                pass
    print(f"\nSummary: {extracted} extracted, {failed} failed")
else:
    print("No .rar files found in demos folder")

No .rar files found in demos folder


In [115]:
# Check .dem files with 0 byte size and remove them
dem_files = glob(os.path.join(constants.DEMOS_DIR, "**", "*.dem"), recursive=True)
zero_byte_files = [f for f in dem_files if os.path.getsize(f) == 0]
if zero_byte_files:
    print(f"Found {len(zero_byte_files)} zero-byte .dem files. Removing them...")
    for file in zero_byte_files:
        try:
            os.remove(file)
            print(f"  🗑️ Removed: {os.path.basename(file)}")
        except Exception as e:
            print(f"  ⚠ Failed to remove {os.path.basename(file)}: {str(e)[:50]}")
else:
    print("No zero-byte .dem files found")

No zero-byte .dem files found


## Data & Assets Paths
Defines locations for demos and map backgrounds used later.

In [ ]:
# Define the path to the 'demos' folder
DEMOS_FOLDER = os.path.join(os.path.dirname(''), "..", "demos")

ASSETS_FOLDER = os.path.join(os.path.dirname(''), "..", "assets")
MAPS_BACKGROUND_FOLDER = os.path.join(ASSETS_FOLDER, "maps_background")

# Load all demos paths
demos_paths = glob(os.path.join(DEMOS_FOLDER, "**", "*.dem"), recursive=True)
print(f"Found {len(demos_paths)} demo files. Listing first 5: {demos_paths[:5]}")

# Load all map background images paths
maps_background_paths = {f.split('.')[0]: os.path.join(MAPS_BACKGROUND_FOLDER, f) for f in os.listdir(MAPS_BACKGROUND_FOLDER) if f.endswith('.png')}

Found 4 demo files. Listing first 5: ['../s_demos/furia-vs-natus-vincere-m1-nuke.dem', '../s_demos/natus-vincere-vs-pain-m2-mirage.dem', '../s_demos/natus-vincere-vs-pain-m1-anubis.dem', '../s_demos/furia-vs-natus-vincere-m2-mirage.dem']


# Parse Demos

## Parse Header and Ticks
Reads demo header and parses full tick stream with desired properties.

In [117]:
# Parse a demo file
def parse_demo(demo_path: str):
    try:
        parser = DemoParser(demo_path=demo_path)
        header = parser.parse_header()
        header['demo_path'] = demo_path
        header['map_png_path'] = maps_background_paths.get(header['map_name'], None)
        ticks_df = parser.parse_ticks(wanted_props=['tick', 'X', 'Y', 'health', 'weapon_name', 'is_freeze_period', 'is_warmup_period','team_name', 'round_win_status', 'round_win_reason', 'bomb_planted', 'round_start_time',
        'round_end_time', 'is_bomb_planted', 'game_time', 'total_rounds_played', 'current_equip_value'])
        ticks_df.sort_values(['total_rounds_played', 'tick', 'team_name'], inplace=True)
        return ticks_df, header
    except Exception as e:
        return None, None

## Aggregate Round Outcomes
Compute per-round winner and reason using the last non-zero status/reason across the full tick stream.

In [118]:
def process_round_results(ticks_df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the ticks dataframe to extract round results including winners and reasons.

    Args:
        ticks_df (pd.DataFrame): DataFrame containing tick-level data with round information.

    Returns:
        pd.DataFrame: DataFrame with round index, winner, and reason for each round.
    """
    # Capture full tick stream (including freeze/warmup) to evaluate round winners
    ticks_all = ticks_df.copy()
    
    # Filter out warmup ticks if available to avoid buggy warmup winners
    if 'is_warmup_period' in ticks_all.columns:
        ticks_all = ticks_all[ticks_all['is_warmup_period'] == False].copy()

    def last_non_zero(series):
        non_zero = series[series != 0]
        return int(non_zero.iloc[-1]) if not non_zero.empty else 0

    # Important: the win event appears after total_rounds_played increments.
    # Attribute win status/reason to the previous round index.
    ticks_all['round_for_outcome'] = ticks_all['total_rounds_played']
    win_mask = (ticks_all['round_win_status'] != 0) | (ticks_all['round_win_reason'] != 0)
    ticks_all.loc[win_mask, 'round_for_outcome'] = ticks_all.loc[win_mask, 'round_for_outcome'] - 1

    round_results = (
        ticks_all.groupby('round_for_outcome')
        .agg(round_winner=('round_win_status', last_non_zero),
            round_reason=('round_win_reason', last_non_zero))
        .reset_index()
        .rename(columns={'round_for_outcome': 'round_index'})
    )

    # Keep only rounds with a detected winner
    round_results = round_results[round_results['round_winner'] != 0].copy()

    # Backfill: ensure the final observed round has an outcome if a final win event exists
    try:
        last_round = int(ticks_df['total_rounds_played'].max())
        if last_round not in set(round_results['round_index'].tolist()):
            last_event = ticks_all.loc[win_mask].tail(1)
            if not last_event.empty:
                inferred_round = int(last_event['total_rounds_played'].iloc[0] - 1)
                if inferred_round == last_round:
                    rr = int(last_event['round_win_reason'].iloc[0]) if int(last_event['round_win_reason'].iloc[0]) != 0 else 0
                    rw = int(last_event['round_win_status'].iloc[0]) if int(last_event['round_win_status'].iloc[0]) != 0 else 0
                    if rw != 0:
                        round_results = pd.concat([round_results, pd.DataFrame([{'round_index': last_round, 'round_winner': rw, 'round_reason': rr}])], ignore_index=True)
    except Exception as _e:
        pass

    return round_results

## Attach Outcomes and Normalize Rounds
Merge round outcomes into ticks, drop non-official rounds, and remap rounds to be contiguous.

In [119]:
def integrate_round_results(ticks_df: pd.DataFrame, round_results: pd.DataFrame) -> pd.DataFrame:
    # Join with round-level outcomes computed before filtering
    ticks_df = ticks_df.merge(round_results, left_on='total_rounds_played', right_on='round_index', how='left')

    # Drop non-official rounds (e.g., warmup)
    ticks_df = ticks_df.dropna(subset=['round_winner']).copy()
    ticks_df['round_winner'] = ticks_df['round_winner'].astype('int32')
    ticks_df['round_reason'] = ticks_df['round_reason'].fillna(0).astype('int32')

    # Replace per-tick status/reason with aggregated round results
    ticks_df['round_win_status'] = ticks_df['round_winner']
    ticks_df['round_win_reason'] = ticks_df['round_reason']

    # Clean helper columns
    ticks_df.drop(columns=['round_winner', 'round_reason', 'round_index'], inplace=True)
    return ticks_df

## Compute Time Features
Derive `seconds_elapsed_in_round` from `game_time - round_start_time`.

In [120]:
def finalize_ticks_dataframe(ticks_df: pd.DataFrame) -> pd.DataFrame:
    # Seconds elapsed in round
    ticks_df['seconds_elapsed_in_round'] = (ticks_df['game_time'] - ticks_df['round_start_time']).clip(lower=0)
    ticks_df.drop(columns=['round_start_time', 'game_time'], inplace=True)
    return ticks_df

## Drop Pre-Freeze Prologue
Ignore the very first ticks before freeze time is active; keep buy (freeze) and live phases.

In [121]:
def filter_initial_round_ticks(ticks_df: pd.DataFrame) -> pd.DataFrame:
    # Ignore initial ticks before the round's freeze time begins (keep freeze/buy + live phase)
    # 1) Find the first tick in each round where freeze is flagged (if available)
    if 'is_freeze_period' in ticks_df.columns:
        freeze_start = (
            ticks_df.loc[ticks_df['is_freeze_period'] == True, ['total_rounds_played', 'tick']]
                .groupby('total_rounds_played', as_index=False)['tick']
                .min()
                .rename(columns={'tick': 'freeze_start_tick'})
        )
    else:
        import pandas as pd
        freeze_start = pd.DataFrame(columns=['total_rounds_played','freeze_start_tick'])

    # 2) Fallback: earliest tick where seconds_elapsed_in_round > 1
    if 'seconds_elapsed_in_round' in ticks_df.columns:
        spawn_defined = (
            ticks_df.loc[ticks_df['seconds_elapsed_in_round'] > 1, ['total_rounds_played', 'tick']]
                .groupby('total_rounds_played', as_index=False)['tick']
                .min()
                .rename(columns={'tick': 'spawn_defined_tick'})
        )
    else:
        import pandas as pd
        spawn_defined = pd.DataFrame(columns=['total_rounds_played','spawn_defined_tick'])

    # 3) Build per-round cutoff tick: prefer freeze_start_tick, else spawn_defined_tick
    cutoffs = freeze_start.merge(spawn_defined, on='total_rounds_played', how='outer')
    cutoffs['keep_from_tick'] = cutoffs['freeze_start_tick'].fillna(cutoffs['spawn_defined_tick'])

    # 4) Join and filter ticks_df to only keep ticks at/after cutoff
    ticks_df = ticks_df.merge(cutoffs[['total_rounds_played', 'keep_from_tick']], on='total_rounds_played', how='left')
    # If a round still lacks a cutoff, default to that round's first tick
    ticks_df['keep_from_tick'] = ticks_df['keep_from_tick'].fillna(ticks_df.groupby('total_rounds_played')['tick'].transform('min'))
    ticks_df = ticks_df[ticks_df['tick'] >= ticks_df['keep_from_tick']].copy()
    ticks_df.drop(columns=['keep_from_tick'], inplace=True)
    return ticks_df

# Normalize data

In [122]:
def set_categorical_data_types(ticks_df: pd.DataFrame) -> pd.DataFrame:
    # Set categorical data types
    ticks_df['weapon_name'] = ticks_df['weapon_name'].astype('category')
    ticks_df['team_name'] = ticks_df['team_name'].astype('category')
    return ticks_df

# Dataframe with the summary of each round

 Create a dataframe to make the model more lightweight. Dataframe should calculate:
  - Round
  - Team A Players
  - Team B Players
  - Team A Weapons
  - Team B Weapons

In [123]:
def build_round_summary(ticks_df: pd.DataFrame, round_results: pd.DataFrame) -> pd.DataFrame:
    """
    Build a lightweight per-round summary DataFrame from tick-level data.

    Args:
        ticks_df (pd.DataFrame): DataFrame containing tick-level data.
        round_results (pd.DataFrame): DataFrame containing round outcomes.

    Returns:
        pd.DataFrame: Summary DataFrame with players and weapons by team for each round.
    """
    # Build a lightweight per-round summary: players and weapons by team
    def unique_list(series, exclude=(None, 'None', '')):
        if series is None:
            return []
        s = series.dropna()
        vals = [v for v in s.tolist() if v not in exclude]
        # preserve order of first appearance
        return list(dict.fromkeys(vals))

    team_ct_label, team_t_label = 'CT', 'TERRORIST'
    player_col = 'steamid' if 'steamid' in ticks_df.columns else ('name' if 'name' in ticks_df.columns else None)

    # Helper: first non-null value
    def first_non_null(series):
        s = series.dropna()
        return s.iloc[0] if not s.empty else None

    # Determine the first post-freeze tick per round (fallback to time >1s, else earliest)
    import pandas as pd
    rounds_df = pd.DataFrame({'total_rounds_played': sorted(ticks_df['total_rounds_played'].unique())})

    min_tick = (
        ticks_df.groupby('total_rounds_played', as_index=False)['tick']
        .min()
        .rename(columns={'tick': 'min_tick'})
    )
    rounds_df = rounds_df.merge(min_tick, on='total_rounds_played', how='left')

    if 'is_freeze_period' in ticks_df.columns:
        live_tick = (
            ticks_df.loc[ticks_df['is_freeze_period'] == False, ['total_rounds_played', 'tick']]
                .groupby('total_rounds_played', as_index=False)['tick']
                .min()
                .rename(columns={'tick': 'live_tick'})
        )
    else:
        live_tick = pd.DataFrame(columns=['total_rounds_played', 'live_tick'])
    rounds_df = rounds_df.merge(live_tick, on='total_rounds_played', how='left')

    if 'seconds_elapsed_in_round' in ticks_df.columns:
        spawn_tick = (
            ticks_df.loc[ticks_df['seconds_elapsed_in_round'] > 1, ['total_rounds_played', 'tick']]
                .groupby('total_rounds_played', as_index=False)['tick']
                .min()
                .rename(columns={'tick': 'spawn_tick'})
        )
    else:
        spawn_tick = pd.DataFrame(columns=['total_rounds_played', 'spawn_tick'])
    rounds_df = rounds_df.merge(spawn_tick, on='total_rounds_played', how='left')

    rounds_df['first_tick'] = rounds_df['live_tick'].fillna(rounds_df['spawn_tick']).fillna(rounds_df['min_tick'])

    # Keep only rows at the first post-freeze tick per round
    first_ticks = ticks_df.merge(rounds_df[['total_rounds_played', 'first_tick']], on='total_rounds_played', how='left')
    first_ticks = first_ticks[first_ticks['tick'] == first_ticks['first_tick']].copy()

    # Build safe maps for round outcomes to avoid IndexError when missing
    winners_map = {}
    reasons_map = {}
    if 'round_index' in round_results.columns:
        winners_map = round_results.set_index('round_index')['round_winner'].to_dict()
        reasons_map = round_results.set_index('round_index')['round_reason'].to_dict()

    rows = []
    for r, grp in first_ticks.groupby('total_rounds_played'):
        ct = grp[grp['team_name'] == team_ct_label]
        tt = grp[grp['team_name'] == team_t_label]

        ct_players = unique_list(ct[player_col]) if player_col else []
        tt_players = unique_list(tt[player_col]) if player_col else []

        # Team equip values at the first post-freeze tick, using current_equip_value
        if 'current_equip_value' in ct.columns:
            ct_equip = int(ct['current_equip_value'].dropna().sum())
        else:
            ct_equip = None
        if 'current_equip_value' in tt.columns:
            tt_equip = int(tt['current_equip_value'].dropna().sum())
        else:
            tt_equip = None

        rows.append({
            'total_rounds_played': int(r),
            'round_winner': int(winners_map.get(int(r), 0)),
            'round_reason': int(reasons_map.get(int(r), 0)),
            'team_ct_name': team_ct_label,
            'team_t_name': team_t_label,
            'team_ct_players': ct_players,
            'team_t_players': tt_players,
            'team_ct_current_equip_value': ct_equip,
            'team_t_current_equip_value': tt_equip,
        })

    round_summary_df = pd.DataFrame(rows).sort_values('total_rounds_played').reset_index(drop=True)

    round_summary_df['round'] = round_summary_df['total_rounds_played'] + 1
    round_summary_df.drop(columns=['total_rounds_played'], inplace=True)
    return round_summary_df

# Train

In [124]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from tqdm import tqdm
import pandas as pd
import logging

def clean_demos_safe(demos_paths, max_workers=5, verbose=False):
    """
    Safer clean_demos:
    - Always returns (DataFrame, list)
    - Does not remove files automatically
    - Thread-safe and efficient
    """
    if verbose:
        logging.basicConfig(level=logging.INFO)

    dfs, wrong = [], []
    lock = Lock()

    def _worker(demo_path):
        def fail(): return pd.DataFrame(), [demo_path]
        try:
            ticks_df, header = parse_demo(demo_path)
            map_name = header.get('map_name')
            if ticks_df is None:
                return fail()

            round_results = process_round_results(ticks_df)
            ticks_df = integrate_round_results(ticks_df, round_results)
            ticks_df = finalize_ticks_dataframe(ticks_df)
            ticks_df = filter_initial_round_ticks(ticks_df)
            ticks_df = set_categorical_data_types(ticks_df)

            round_summary_df = build_round_summary(ticks_df, round_results)
            if round_summary_df is None or round_summary_df.empty:
                return fail()
            round_summary_df['map_name'] = map_name

            first = round_summary_df.iloc[0][['team_ct_current_equip_value', 'team_t_current_equip_value']]
            if first.isna().any():
                return fail()

            if first['team_ct_current_equip_value'] <= 5500 and first['team_t_current_equip_value'] <= 5500:
                return round_summary_df, []
            else:
                return fail()

        except Exception as e:
            if verbose:
                logging.exception(f"Error processing {demo_path}")
            return fail()

    with tqdm(total=len(demos_paths), desc="Cleaning Demos") as progress_bar:
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            futures = {ex.submit(_worker, p): p for p in demos_paths}
            for fut in as_completed(futures):
                df_part, wrong_part = fut.result()
                progress_bar.update(1)
                if not df_part.empty:
                    dfs.append(df_part)
                if wrong_part:
                    wrong.extend(wrong_part)

    wrong = list(dict.fromkeys(wrong))  # deduplicate
    final_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    return final_df, wrong

final_df, wrong_demos = clean_demos_safe(demos_paths, max_workers=10, verbose=True)

Cleaning Demos: 100%|██████████| 4/4 [00:21<00:00,  5.25s/it]


In [125]:
# Prepare data
X = final_df.drop(columns=['round_winner', 'team_ct_name', 'team_t_name', 'round_reason'])
y = final_df["round_winner"]

# Split into train and test (hold-out set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [126]:
import pandas as pd
import joblib
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

class TeamIdEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.classes_ = None

    def fit(self, X, y=None):
        # X deve ter duas colunas: ['equipa_1_ids', 'equipa_2_ids']
        all_ids = set()
        for col in X.columns:
            for lista in X[col]:
                all_ids.update(lista)
        self.classes_ = sorted(list(all_ids))
        return self

    def transform(self, X):
        # Criar matriz de zeros (0 = não joga)
        res = np.zeros((X.shape[0], len(self.classes_)), dtype=int)
        id_map = {id_: i for i, id_ in enumerate(self.classes_)}

        for row_idx in range(X.shape[0]):
            # Equipa 1 -> Valor 1
            for user_id in X.iloc[row_idx, 0]:
                if user_id in id_map:
                    res[row_idx, id_map[user_id]] = 1
            # Equipa 2 -> Valor 2
            for user_id in X.iloc[row_idx, 1]:
                if user_id in id_map:
                    res[row_idx, id_map[user_id]] = 2
        return res

    def get_feature_names_out(self, input_features=None):
        return [f"player_{c}" for c in self.classes_]

preprocessor = ColumnTransformer(
    transformers=[
        ('player_id', TeamIdEncoder(), ['team_ct_players', 'team_t_players']),
        ('map_', OneHotEncoder(handle_unknown='ignore'), ['map_name']),
    ], remainder='passthrough'
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', RandomForestClassifier(random_state=0, n_estimators=100))
])
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('player_id', ...), ('map_', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [127]:
X_transformed = pipeline.named_steps['preprocessor'].transform(X)
column_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
pd.DataFrame(X_transformed, columns=column_names)

,player_id__player_76561197960690195,player_id__player_76561198013243326,player_id__player_76561198015308884,player_id__player_76561198050250233,player_id__player_76561198058500492,player_id__player_76561198067763828,player_id__player_76561198074017668,player_id__player_76561198118646644,player_id__player_76561198164970560,player_id__player_76561198176878303,...,player_id__player_76561198350342505,player_id__player_76561198377335846,player_id__player_76561198385657675,player_id__player_76561199063068840,map___map_name_de_anubis,map___map_name_de_mirage,map___map_name_de_nuke,remainder__team_ct_current_equip_value,remainder__team_t_current_equip_value,remainder__round
0,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,4350.0,4250.0,1.0
1,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,21050.0,16550.0,2.0
2,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,24850.0,10250.0,3.0
3,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,26350.0,27750.0,4.0
4,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,28350.0,28750.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,29450.0,27950.0,19.0
70,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,11850.0,28150.0,20.0
71,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,24850.0,27650.0,21.0
72,1.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,28100.0,25100.0,22.0


# Prediction

In [132]:
prob = pipeline.predict_proba(X_test)
pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("Probabilities example:", prob[:10])

Accuracy: 0.4
Probabilities example: [[0.59 0.41]
 [0.15 0.85]
 [0.03 0.97]
 [0.85 0.15]
 [0.34 0.66]
 [0.45 0.55]
 [0.34 0.66]
 [0.78 0.22]
 [0.51 0.49]
 [0.78 0.22]]


In [141]:
from codecs import ignore_errors
from django.test import ignore_warnings
from lazypredict.Supervised import LazyClassifier

# 1. Pré-processar os dados manualmente com o teu ColumnTransformer
# (Isto transforma as tuas listas de IDs em colunas 0, 1, 2)
X_transformed = preprocessor.fit_transform(X)

# 2. Dividir os dados já transformados
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2)

# 3. Correr o LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 30, number of negative: 29
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 118
[LightGBM] [Info] Number of data points in the train set: 59, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508475 -> initscore=0.033902
[LightGBM] [Info] Start training from score 0.033902
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [116]:
import joblib

# Save the trained model
joblib.dump(rf, 'round_winner_model.pkl')
joblib.dump(all_players, 'all_players.pkl')
joblib.dump(final_df['map_name'].unique().tolist(), 'maps_names.pkl')

['maps_names.pkl']